In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [3]:
from langchain.prompts import load_prompt, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

In [46]:
template = '''
Generate search queries to research material for post.
Generate search queries based on the purpose and table of contents of the article given as input.
Each search query will be used to search for data on Google, YouTube, etc.
Each search query consists of a few words.
Just say the search query. But if you don't have any, don't say anything and don't explain.
Each search query is in no particular order and is separated by commas.
Answer in Korean.

Purpose of the post:
===
{purpose}
===

table of contents:
===
{table}
===

keyword quries:
'''

In [55]:
class KeywordsChain:
    def __init__(self, keywords_template=None, keywords_template_path='../openai_skt/models/templates/keywords_prompt.json', verbose=False) -> None:
        if keywords_template is not None:
            self.keywords_prompt = PromptTemplate(
                    input_variables=["purpose","table"],
                    template=keywords_template,
                )
        else:
            self.keywords_template_path = keywords_template_path
            self.keywords_prompt = load_prompt(self.keywords_template_path)
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        self.verbose = verbose
        self.keywords_chain = LLMChain(llm=self.llm, prompt=self.keywords_prompt, verbose=self.verbose)

    def run(self, purpose:str=None, table:str=None):
        input_dict = {'purpose': purpose, 'table' : table}
        keywords = self.keywords_chain.run(input_dict)
        return keywords

    async def arun(self, purpose:str=None):
        input_dict = {'purpose': purpose}
        keywords = await self.keywords_chain.arun(input_dict)
        return keywords

In [56]:
keywords_chain = KeywordsChain(keywords_template=template, verbose=True)

In [57]:
purpose = '저출산 대응·소비 진작…"결혼자금 증여세 면제 확대"는 어떤가'
table_contents = '''1. 서론
- 저출산 문제와 결혼자금 증여세 면제의 관계
- 결혼자금 증여세 면제 확대에 대한 간략한 소개
2. 결혼자금 증여세 면제의 현황
- 현재의 결혼자금 증여세 면제 정책
- 이전의 결혼자금 증여세 면제 정책과의 비교
3. 결혼자금 증여세 면제 확대의 효과
- 저출산 대응을 위한 결혼자금 증여세 면제의 효과
- 소비 활성화에 미치는 영향
4. 결혼자금 증여세 면제 확대의 문제점
- 예산 문제와 재분배의 공정성 문제
- 기타 부작용과 부작용 완화 방안
5. 결론
- 결혼자금 증여세 면제 확대의 장단점과 대안
- 글쓰기 주제와 관련된 개인적인 견해나 생각
'''

In [58]:
result = keywords_chain.run(purpose=purpose, table=table_contents)



> Entering new LLMChain chain...
Prompt after formatting:

Generate search queries to research material for post.
Generate search queries based on the purpose and table of contents of the article given as input.
Each search query will be used to search for data on Google, YouTube, etc.
Each search query consists of a few words.
Just say the search query. But if you don't have any, don't say anything and don't explain.
Each search query is in no particular order and is separated by commas.
Answer in Korean.

Purpose of the post:
===
저출산 대응·소비 진작…"결혼자금 증여세 면제 확대"는 어떤가
===

table of contents:
===
1. 서론
- 저출산 문제와 결혼자금 증여세 면제의 관계
- 결혼자금 증여세 면제 확대에 대한 간략한 소개
2. 결혼자금 증여세 면제의 현황
- 현재의 결혼자금 증여세 면제 정책
- 이전의 결혼자금 증여세 면제 정책과의 비교
3. 결혼자금 증여세 면제 확대의 효과
- 저출산 대응을 위한 결혼자금 증여세 면제의 효과
- 소비 활성화에 미치는 영향
4. 결혼자금 증여세 면제 확대의 문제점
- 예산 문제와 재분배의 공정성 문제
- 기타 부작용과 부작용 완화 방안
5. 결론
- 결혼자금 증여세 면제 확대의 장단점과 대안
- 글쓰기 주제와 관련된 개인적인 견해나 생각

===

keyword quries:


> Finished chain.


In [54]:
print(result)

- 저출산 대응 정책
- 결혼자금 증여세 면제 현황
- 결혼자금 증여세 면제 정책 비교
- 결혼자금 증여세 면제의 효과
- 저출산 대응을 위한 결혼자금 증여세 면제
- 결혼자금 증여세 면제와 소비 활성화
- 결혼자금 증여세 면제 확대의 문제점
- 예산 문제와 결혼자금 증여세 면제의 공정성
- 결혼자금 증여세 면제의 부작용과 완화 방안
- 결혼자금 증여세 면제 확대의 장단점
- 결혼자금 증여세 면제 대안
- 결혼자금 증여세 면제에 대한 개인적인 견해나 생각
